In [1]:
import pandas as pd
        
ISW = pd.read_csv('ISW.csv')
ISW

,date,title,full_url,main_html,main_html_v2,main_text
0,2022-02-25,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...",['Russian forces carried out additional air an...
1,2022-02-26,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...","['Key Takeaways', 'Russia has surprisingly fai..."
2,2022-02-27,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p style=""text-align: left;""><strong><span st...",['Russian forces largely conducted an operatio...
3,2022-02-28,"Russian Offensive Campaign Assessment, Februar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...",['The Russian military has begun using area-at...
4,2022-03-01,"Russian Offensive Campaign Assessment, March 1",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...",['The Russian military has continued using are...
...,...,...,...,...,...,...
324,2023-01-21,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...",['Ukrainian forces have previously employed a ...
325,2023-01-22,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...",['ISW is publishing an abbreviated campaign up...
326,2023-01-23,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...",['Ukrainian intelligence assessed that Russian...
327,2023-01-24,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...",['Western states’ provision of main battle tan...


In [5]:

from nltk.corpus import stopwords
import spacy
import re 
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_sm")
# lemmatizing
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text



# lowercasing
def lowercase_text(text):
    # sample = ast.literal_eval(text)
    return [i.lower() for i in text]

# text cleaning from odd elements
def clean_text(text_list):
    # Ensure input is a list of strings
    if not all(isinstance(text, str) for text in text_list):
        raise ValueError("Input must be a list of strings")

    cleaned_text = ' '.join(text_list)

    # Perform text cleaning operations
    cleaned_text = re.sub(
        r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\b',
        lambda match: match.group(0).lower(), cleaned_text, flags=re.IGNORECASE)

    cleaned_text = re.sub(r'[-/\\—\xa0"’“]', ' ', cleaned_text)
    cleaned_text = re.sub(r'[’“]', ' ', cleaned_text)
    cleaned_text = re.sub(r'(march)', ' ', cleaned_text)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\b(?:pm|am)\b', '', cleaned_text)
    cleaned_text = re.sub(r'\b(?:\d+(?:st|nd|rd|th)?)\b', '', cleaned_text)
    cleaned_text = re.sub(r'\b\d+[a-zA-Z]+|(?<!\d)[a-zA-Z]+\d+\b', '', cleaned_text)
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text.strip())

    return cleaned_text

# stopword removal
def remove_stopwords(text):
    ", ".join(stopwords.words('english'))
    STOPWORDS = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

pure = ISW 

# use lowercasing
pure["main_text"] = pure["main_text"].apply(lowercase_text)

# clean text
pure['new'] = pure['main_text'].apply(lambda x: clean_text(x))
pure["main_text"] = pure["new"]
pure.drop(["new"], axis=1)

# remove stopwords
pure["text_wo_stop"] = pure["main_text"].apply(lambda text: remove_stopwords(text))

# lemmatizing
pure['text_for_vect2'] = pure['text_wo_stop'].apply(lemmatize_text)

# dropping out all useless columns
pure.drop(["text_wo_stop", "main_text"], axis=1)

# vectorizing
#ENTER THE RIGHT PATH

# with open("/Users/ivantyshchenko/Documents/GitHub/4-pythonic-squad/source/NLP/tfidf.pkl", "rb") as f:

tfidf_vect = TfidfVectorizer(max_features=700)
tfidf_matrix = tfidf_vect.transform(pure['text_for_vect2'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
vect_df = pd.concat([pure["date"], tfidf_df], axis=1)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'en_core_web_sm' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg

In [6]:
!pip install nltk

!pip install spacy==2.3.5

!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

!pip install pyresparser

  Using cached spacy-2.3.5.tar.gz (5.8 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [298 lines of output]
        Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
        Using cached Cython-3.0.10-py2.py3-none-any.whl.metadata (3.2 kB)
        Using cached cymem-2.0.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.4 kB)
        Using cached preshed-3.0.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Using cached murmurhash-1.0.10-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.0 kB)
        Using cached thinc-7.4.6.tar.gz (1.3 MB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'error'
        error: subprocess-exited-with-error
      
  